In [1]:
import pandas as pd
import glob
import numpy as np
from tqdm import tqdm
import imblearn
import tensorflow_addons as tfa
from utils.model_1 import *
from tensorflow import keras
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
training_val_combined=pd.read_parquet('combined_sequence_sampled/training.gzip', engine='auto')
training_val_combined['labels']=training_val_combined['labels'].astype(int)
training_val_combined['measurements']=training_val_combined['measurements'].apply(lambda x: np.vstack(x))
X=list(training_val_combined['measurements'])
X_padded=tf.keras.utils.pad_sequences(X,dtype=object)
y=list(training_val_combined['labels'])
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
p=list(Counter(y).values())
weight_for_0 = (1 / p[0]) * (len(y) / 2.0)
weight_for_1 = (1 / p[1]) * (len(y) / 2.0)
weight_for_2 = (1 / p[2]) * (len(y) / 2.0)
weight_for_3 = (1 / p[3]) * (len(y) / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1,2: weight_for_2, 3: weight_for_3}
y_padded=np.array(y)
print(class_weight)
idx = np.random.permutation(len(y))
X_balanced_reshaped = X_padded[idx]
y_balanced_reshaped = y_padded[idx].reshape(-1,1)
num_classes=len(np.unique(y_balanced_reshaped))
encoder = OneHotEncoder()
import datetime
# transform data
onehot = encoder.fit_transform(y_balanced_reshaped)
y_balanced_reshaped_onehot=onehot.toarray()
#X_padded=tf.keras.utils.pad_sequences(X,dtype=object)
X_balanced_reshaped=tf.convert_to_tensor(X_balanced_reshaped, dtype=tf.float64)
y_balanced_reshaped_onehot=tf.convert_to_tensor(y_balanced_reshaped_onehot)
METRICS = [
      keras.metrics.CategoricalAccuracy(name='Accuracy',dtype=tf.float32),
    keras.metrics.Precision(name='Precision'),
        keras.metrics.Precision(name='Recall'),
]
%load_ext tensorboard

/Users/mfarooq2/miniconda3/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


{0: 0.6662264675510366, 1: 12.142055736518277, 2: 9.176285557986871, 3: 3.2501937608990503}
Metal device set to: Apple M1


In [16]:
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    X, padding="post")

In [17]:
padded_inputs.shape

(67097, 20, 6)

In [10]:
logdir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()

def lr_schedule(epoch):
  """
  Returns a custom learning rate that decreases as epochs progress.
  """
  learning_rate = 0.2
  if epoch > 10:
    learning_rate = 0.02
  if epoch > 20:
    learning_rate = 0.01
  if epoch > 50:
    learning_rate = 0.005

  tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
  return learning_rate
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [5]:
%tensorboard --port 6009 --logdir logs/scalars

In [11]:
model=LSTM_Model(X_balanced_reshaped.shape[1:],num_classes)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 20, 100)           42800     
                                                                 
 dropout_5 (Dropout)         (None, 20, 100)           0         
                                                                 
 lstm_5 (LSTM)               (None, 20, 100)           80400     
                                                                 
 dropout_6 (Dropout)         (None, 20, 100)           0         
                                                                 
 lstm_6 (LSTM)               (None, 20, 100)           80400     
                                                                 
 dropout_7 (Dropout)         (None, 20, 100)           0         
                                                                 
 lstm_7 (LSTM)               (None, 100)              

In [12]:
lr_callback = keras.callbacks.LearningRateScheduler(lr_schedule)

In [13]:
epochs = 50
batch_size = 1000
callbacks = [keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.00001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=0),tensorboard_callback,lr_callback,
]
opt = keras.optimizers.legacy.Adam()
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy",
    metrics=METRICS,
)

In [ ]:
y_balanced_reshaped_onehot.shape

In [ ]:
batch_size

In [14]:
history = model.fit(
    X_balanced_reshaped,
    y_balanced_reshaped_onehot,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=0,class_weight=class_weight)

2023-04-22 18:35:49.501156: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [ ]:
plt.plot(history.history['loss'],label='val_loss')
#plt.plot(history.history['val_precision'],label='val_precision')
#plt.plot(history.history['val_recall'],label='val_recall')
#plt.plot(history.history['val_loss'],label='val_loss')
plt.legend()

In [ ]:
model.save('models2/')

In [ ]:





training_history = model.fit(
    x_train, # input
    y_train, # output
    batch_size=train_size,
    verbose=0, # Suppress chatty output; use Tensorboard instead
    epochs=100,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback, lr_callback],
)